In [34]:
import pymongo

In [35]:
client = pymongo.MongoClient("localhost:27017")
db = client['bankDB']
collection = db['BankData']

In [36]:
collection.find_one()

{'_id': ObjectId('66c45a1fe5b03eae919eac97'),
 'age': 58,
 'job': 'management',
 'marital': 'married',
 'education': 'tertiary',
 'default': 'no',
 'balance': 2143,
 'housing': 'yes',
 'loan': 'no',
 'contact': 'unknown',
 'day': 5,
 'month': 'may',
 'duration': 261,
 'campaign': 1,
 'pdays': -1,
 'previous': 0,
 'poutcome': 'unknown',
 'y': 'no'}

#### 1. Give marketing success rate (No. of people subscribed / total no. of entries)

In [37]:
print("subscribed:", collection.count_documents({"y": "yes"}))
print("total:", collection.count_documents({}))


print("success_rate:", collection.count_documents({"y": "yes"}) / collection.count_documents({}) * 100)

subscribed: 5289
total: 45211
success_rate: 11.698480458295547


#### 2. Give marketing failure rate

In [38]:
collection.count_documents({"y": "no"}) / collection.count_documents({}) * 100

88.30151954170445

In [39]:
query2 = [
    {
        "$facet": {
            "totalCount": [{"$count": "total"}],
            "subscribed": [{"$match": {"y": "yes"}},
                          {"$count": "subscribed"}]
        }
    },
    {
        "$project": {
            "totalCount": {"$arrayElemAt":["totalCount.total", 0]},
            "subscribedCount": {"$arrayElemAt":["totalCount.subscribed", 0]},
            "successsRate": {"$divide": ["subscribedCount", "totalCount"]}
        }
    }
]

query3 = [
    {
        "$facet": {
            "totalCount": [{"$count": "total"}],
            "subscribed": [
                {"$match": {"y": "yes"}},
                {"$count": "subscribed"}
            ]
        }
    },
    {
        "$project": {
            "totalCount": {"$arrayElemAt": ["$totalCount.total", 0]},
            "subscribedCount": {"$arrayElemAt": ["$subscribed.subscribed", 0]},
            "successRate": { "$divide": [
                {"$arrayElemAt": ["$subscribed.subscribed", 0]}, 
                { "$arrayElemAt": [ "$totalCount.total", 0]}
            ]}
        }
    }
]


result = collection.aggregate(query3)

list(result)[0]

{'totalCount': 45211,
 'subscribedCount': 5289,
 'successRate': 0.11698480458295547}

In [40]:
query3 = [
    {
        "$facet": {
            "totalCount": [{"$count": "total"}],
            "subscribed": [
                {"$match": {"y": "no"}},
                {"$count": "subscribed"}
            ]
        }
    },
    {
        "$project": {
            "totalCount": {"$arrayElemAt": ["$totalCount.total", 0]},
            "nonSubscribedCount": {"$arrayElemAt": ["$subscribed.subscribed", 0]},
            "failureRate": { "$divide": [
                {"$arrayElemAt": ["$subscribed.subscribed", 0]}, 
                { "$arrayElemAt": [ "$totalCount.total", 0]}
            ]}
        }
    }
]

result = collection.aggregate(query3)

list(result)[0]

{'totalCount': 45211,
 'nonSubscribedCount': 39922,
 'failureRate': 0.8830151954170445}

#### 3. Give the maximum, mean, median, minimum age of the average targeted customer.

In [43]:
query = [
    {'$match': {'y': 'yes'}},
    {'$group': {
            '_id': '$y',
            'maxAge': { '$max': '$age' },
            'meanAge': { '$avg': '$age' },
            'minAge': { '$min': '$age' },
            'ages': { '$push': '$age' }
    }},
    {'$project': {
            'maxAge': 1,
            'meanAge': 1,
            'minAge': 1,
            'ages': 1
    }}
]

result = collection.aggregate(query)

result = list(result)[0]


print(f"Maximum Age: {result['maxAge']}")

print(f"Mean Age: {result['meanAge']:.2f}")

print(f"Minimum Age: {result['minAge']}")

Maximum Age: 95
Mean Age: 41.67
Minimum Age: 18


In [ ]:
query = [
    {'$match': {'y': 'yes'}}, 
    {'$group': {
        '_id': None,         
        'ages': {'$push': '$age'} 
    }}
]

cursor = collection.aggregate(query)

result = list(cursor)[0]

medianAge = sorted(result['ages'])[len(result['ages']) // 2]

print(f"Maximum Age (using python): {max(result['ages'])}")

print(f"Mean Age: {sum(result['ages']) / len(result['ages']):.2f}")

print(f"Median Age: {medianAge}")

print(f"Minimum Age (using python): {min(result['ages'])}")

- median without python

In [45]:
query = [
    {'$match': {'y': 'yes'}},
    {'$group': {
            '_id': '$y',
            'maxAge': { '$max': '$age' },
            'meanAge': { '$avg': '$age' },
            'minAge': { '$min': '$age' },
            'ages': { '$push': '$age' },
    }},
    {'$project': {
            'maxAge': 1,
            'meanAge': 1,
            'minAge': 1,
            'ages': 1,
            'medianAge': {"$median": {"input": "$ages", "method": "approximate"}}
    }}
    
]


result = collection.aggregate(query)

result = list(result)[0]

print(f"Maximum Age: {result['maxAge']}")

print(f"Mean Age: {result['meanAge']:.2f}")

print(f"Median Age: {result['medianAge']}")

print(f"Minimum Age: {result['minAge']}")


Maximum Age: 95
Mean Age: 41.67
Median Age: 38.0
Minimum Age: 18


#### 4. Check if age matters in marketing subscription for deposit

In [46]:
query_4 = [
    {"$match": {"y": "yes"}},
    {"$group": { 
        "_id": "$age", 
        "count": {"$sum":1}
    }},
    {"$sort": {"count":  -1}}
]

result_4 = collection.aggregate(query_4)

list(result_4)

[{'_id': 32, 'count': 221},
 {'_id': 30, 'count': 217},
 {'_id': 33, 'count': 210},
 {'_id': 35, 'count': 209},
 {'_id': 31, 'count': 206},
 {'_id': 34, 'count': 198},
 {'_id': 36, 'count': 195},
 {'_id': 29, 'count': 171},
 {'_id': 37, 'count': 170},
 {'_id': 28, 'count': 162},
 {'_id': 38, 'count': 144},
 {'_id': 39, 'count': 143},
 {'_id': 27, 'count': 141},
 {'_id': 26, 'count': 134},
 {'_id': 41, 'count': 120},
 {'_id': 46, 'count': 118},
 {'_id': 40, 'count': 116},
 {'_id': 25, 'count': 113},
 {'_id': 47, 'count': 113},
 {'_id': 42, 'count': 111},
 {'_id': 45, 'count': 106},
 {'_id': 43, 'count': 103},
 {'_id': 49, 'count': 101},
 {'_id': 60, 'count': 98},
 {'_id': 44, 'count': 93},
 {'_id': 59, 'count': 88},
 {'_id': 53, 'count': 85},
 {'_id': 52, 'count': 85},
 {'_id': 54, 'count': 84},
 {'_id': 48, 'count': 82},
 {'_id': 57, 'count': 78},
 {'_id': 51, 'count': 77},
 {'_id': 55, 'count': 76},
 {'_id': 50, 'count': 72},
 {'_id': 58, 'count': 72},
 {'_id': 24, 'count': 68},
 {'_i

#### 5. Check if marital status mattered for a subscription to deposit

In [47]:
# marital status and subscribed customers relation
query_5 = [
    {"$match": {"y": "yes"}},
    {"$group": { 
        "_id": "$marital", 
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

result_5 = collection.aggregate(query_5)

list(result_5)

[{'_id': 'married', 'count': 2755},
 {'_id': 'single', 'count': 1912},
 {'_id': 'divorced', 'count': 622}]

#### 6. Check if age and marital status together mattered for a subscription to deposit scheme

In [48]:
query_6 = [
    {"$match": {"y": "yes"}},
    {"$group": { 
        "_id": {
            "age": "$age",  
            "marital": "$marital"
        },
        "count": {"$sum":1}
    }},
    {"$sort": {"count":  -1}}
]

result_6 = collection.aggregate(query_6)

list(result_6)

[{'_id': {'age': 30, 'marital': 'single'}, 'count': 151},
 {'_id': {'age': 28, 'marital': 'single'}, 'count': 138},
 {'_id': {'age': 29, 'marital': 'single'}, 'count': 133},
 {'_id': {'age': 32, 'marital': 'single'}, 'count': 124},
 {'_id': {'age': 26, 'marital': 'single'}, 'count': 121},
 {'_id': {'age': 34, 'marital': 'married'}, 'count': 118},
 {'_id': {'age': 31, 'marital': 'single'}, 'count': 111},
 {'_id': {'age': 27, 'marital': 'single'}, 'count': 110},
 {'_id': {'age': 35, 'marital': 'married'}, 'count': 101},
 {'_id': {'age': 36, 'marital': 'married'}, 'count': 100},
 {'_id': {'age': 25, 'marital': 'single'}, 'count': 99},
 {'_id': {'age': 37, 'marital': 'married'}, 'count': 98},
 {'_id': {'age': 33, 'marital': 'married'}, 'count': 97},
 {'_id': {'age': 33, 'marital': 'single'}, 'count': 97},
 {'_id': {'age': 39, 'marital': 'married'}, 'count': 87},
 {'_id': {'age': 32, 'marital': 'married'}, 'count': 87},
 {'_id': {'age': 38, 'marital': 'married'}, 'count': 86},
 {'_id': {'ag

#### 7. Find All Records Where Housing Loan is Approved

In [49]:
# marital status and subscribed customers relation
result_7 = collection.find({"housing": "yes"})

list(result_7)

[{'_id': ObjectId('66c45a1fe5b03eae919eac97'),
  'age': 58,
  'job': 'management',
  'marital': 'married',
  'education': 'tertiary',
  'default': 'no',
  'balance': 2143,
  'housing': 'yes',
  'loan': 'no',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
  'duration': 261,
  'campaign': 1,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a1fe5b03eae919eac98'),
  'age': 44,
  'job': 'technician',
  'marital': 'single',
  'education': 'secondary',
  'default': 'no',
  'balance': 29,
  'housing': 'yes',
  'loan': 'no',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
  'duration': 151,
  'campaign': 1,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a1fe5b03eae919eac99'),
  'age': 33,
  'job': 'entrepreneur',
  'marital': 'married',
  'education': 'secondary',
  'default': 'no',
  'balance': 2,
  'housing': 'yes',
  'loan': 'yes',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
 

#### 8. Find Records with High Account Balance

In [50]:
result_8 = collection.find().sort(
  {"balance": -1}
  ).limit(20)

list(result_8)

[{'_id': ObjectId('66c45a25e5b03eae919f48cc'),
  'age': 51,
  'job': 'management',
  'marital': 'single',
  'education': 'tertiary',
  'default': 'no',
  'balance': 102127,
  'housing': 'no',
  'loan': 'no',
  'contact': 'cellular',
  'day': 3,
  'month': 'jun',
  'duration': 90,
  'campaign': 1,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a23e5b03eae919f130a'),
  'age': 59,
  'job': 'management',
  'marital': 'married',
  'education': 'tertiary',
  'default': 'no',
  'balance': 98417,
  'housing': 'no',
  'loan': 'no',
  'contact': 'telephone',
  'day': 20,
  'month': 'nov',
  'duration': 145,
  'campaign': 5,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a25e5b03eae919f5618'),
  'age': 84,
  'job': 'retired',
  'marital': 'married',
  'education': 'secondary',
  'default': 'no',
  'balance': 81204,
  'housing': 'no',
  'loan': 'no',
  'contact': 'telephone',
  'day': 1,
  'month': 'apr

#### 9. Count Records by Job Type

In [51]:
# marital status and subscribed customers relation
query_9 = [
    {"$group": { 
        "_id": "$job", 
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

result_9 = collection.aggregate(query_9)

list(result_9)

[{'_id': 'blue-collar', 'count': 9732},
 {'_id': 'management', 'count': 9458},
 {'_id': 'technician', 'count': 7597},
 {'_id': 'admin.', 'count': 5171},
 {'_id': 'services', 'count': 4154},
 {'_id': 'retired', 'count': 2264},
 {'_id': 'self-employed', 'count': 1579},
 {'_id': 'entrepreneur', 'count': 1487},
 {'_id': 'unemployed', 'count': 1303},
 {'_id': 'housemaid', 'count': 1240},
 {'_id': 'student', 'count': 938},
 {'_id': 'unknown', 'count': 288}]

#### 10. Count total Married Individuals with Secondary Education who have been approached

In [52]:
query_10 = [
    {"$match": {"education": "secondary", "marital": "married"}},
    {"$group": { 
        "_id": None, 
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

result_10 = collection.aggregate(query_10)

print(list(result_10))

# method 2
print(collection.count_documents({
    "education": "secondary", 
    "marital": "married"}))

[{'_id': None, 'count': 13770}]
13770


#### 11. Calculate the Average Balance for Each Job Type

In [53]:
query_11 = [
    {"$group": { 
        "_id": "$job", 
        "avgBalance": {"$avg": "$balance"}
    }}
]

result_11 = collection.aggregate(query_11)

list(result_11)

[{'_id': 'retired', 'avgBalance': 1984.215106007067},
 {'_id': 'management', 'avgBalance': 1763.6168323112709},
 {'_id': 'blue-collar', 'avgBalance': 1078.8266543362104},
 {'_id': 'technician', 'avgBalance': 1252.6320916151112},
 {'_id': 'services', 'avgBalance': 997.0881078478575},
 {'_id': 'unknown', 'avgBalance': 1772.357638888889},
 {'_id': 'self-employed', 'avgBalance': 1647.9708676377454},
 {'_id': 'entrepreneur', 'avgBalance': 1521.470073974445},
 {'_id': 'unemployed', 'avgBalance': 1521.7459708365311},
 {'_id': 'housemaid', 'avgBalance': 1392.3951612903227},
 {'_id': 'student', 'avgBalance': 1388.0607675906183},
 {'_id': 'admin.', 'avgBalance': 1135.838909301876}]

#### 12. Find the Most Common Education Level Among Those with Loans

In [54]:
query_12 =[
    {"$match": {"loan": "yes"}},
    {"$group": { 
        "_id": "$education", 
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$limit": 1}
]

result_12 = collection.aggregate(query_12)

list(result_12)

[{'_id': 'secondary', 'count': 4303}]

#### 13. Find Individuals with Multiple Campaign Contacts

In [55]:
result_13 = collection.find({"campaign": {"$gt": 1}})

list(result_13)

[{'_id': ObjectId('66c45a1fe5b03eae919eacb3'),
  'age': 46,
  'job': 'management',
  'marital': 'single',
  'education': 'secondary',
  'default': 'no',
  'balance': -246,
  'housing': 'yes',
  'loan': 'no',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
  'duration': 255,
  'campaign': 2,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a1fe5b03eae919eacc0'),
  'age': 50,
  'job': 'management',
  'marital': 'married',
  'education': 'secondary',
  'default': 'no',
  'balance': 49,
  'housing': 'yes',
  'loan': 'no',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
  'duration': 180,
  'campaign': 2,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'},
 {'_id': ObjectId('66c45a1fe5b03eae919eacc6'),
  'age': 44,
  'job': 'technician',
  'marital': 'married',
  'education': 'secondary',
  'default': 'no',
  'balance': 0,
  'housing': 'yes',
  'loan': 'no',
  'contact': 'unknown',
  'day': 5,
  'month': 'may',
  '

#### 14. Determine the Success Rate of the Campaign

In [57]:
# already done

#### 15. Identify Anomalies in Account Balances

In [60]:
query_15 = [
    {"$group": {
        "_id": None, 
        "avgBalance": {"$avg": "$balance"},
        "stdevBalance": {"$stdDevSamp": "$balance"}
    }},
    {"$lookup": {
        "from":  "customers",
        "pipeline": [
            "$project": {
                
            }
        ]
    }}
]

result_15 = collection.aggregate(query_15)

list(result_15)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 10 (495962117.py, line 11)

#### 16. Analyze Seasonal Patterns

In [72]:
query_16 = [
    {"$group": {
        "_id": "$month", 
        "avgDuration": {"$avg": "$duration"},
        "totalCalls" : {"$sum": 1}
    }},
    {"$sort": {"avgDuration": -1, "totalCalls": -1}}
]

result_16 = collection.aggregate(query_16)

list(result_16)

[{'_id': 'dec', 'avgDuration': 329.32710280373834, 'totalCalls': 214},
 {'_id': 'apr', 'avgDuration': 298.09890859481584, 'totalCalls': 2932},
 {'_id': 'sep', 'avgDuration': 292.25215889464596, 'totalCalls': 579},
 {'_id': 'oct', 'avgDuration': 288.30216802168025, 'totalCalls': 738},
 {'_id': 'jan', 'avgDuration': 268.2202423378475, 'totalCalls': 1403},
 {'_id': 'jul', 'avgDuration': 267.9753444525018, 'totalCalls': 6895},
 {'_id': 'may', 'avgDuration': 260.92227226500074, 'totalCalls': 13766},
 {'_id': 'nov', 'avgDuration': 253.14861460957178, 'totalCalls': 3970},
 {'_id': 'feb', 'avgDuration': 248.29822574556437, 'totalCalls': 2649},
 {'_id': 'mar', 'avgDuration': 244.40041928721175, 'totalCalls': 477},
 {'_id': 'jun', 'avgDuration': 243.0878112712975, 'totalCalls': 5341},
 {'_id': 'aug', 'avgDuration': 232.40211301424685, 'totalCalls': 6247}]

#### 17. Determine Correlation Between Loan Status and Balance

In [76]:
query_17 = [
    {"$group": {
        "_id": "$loan", 
        "avgBalance": {"$avg": "$balance"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"avgBalance": -1}}
]

result_17 = collection.aggregate(query_17)

list(result_17)

[{'_id': 'no', 'avgBalance': 1474.4536307846288, 'count': 37967},
 {'_id': 'yes', 'avgBalance': 774.3099116510215, 'count': 7244}]

In [78]:
# housing loan
query_17 = [
    {"$group": {
        "_id": "$housing", 
        "avgBalance": {"$avg": "$balance"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"avgBalance": -1}}
]

result_17 = collection.aggregate(query_17)

list(result_17)

[{'_id': 'no', 'avgBalance': 1596.5012698570788, 'count': 20081},
 {'_id': 'yes', 'avgBalance': 1175.1030640668523, 'count': 25130}]

#### 18. Identify the Most Common Day for Successful Campaigns

In [82]:
query_18 = [
    {"$match": {"y": "yes"}},
    {"$group": {
        "_id": "$day", 
        "successfullCampains": {"$sum": 1}
    }},
    {"$sort": {"successfullCampains": -1}},
    {"$limit": 1}
]

result_18 = collection.aggregate(query_18)

list(result_18)

[{'_id': 30, 'successfullCampains': 271}]

#### 19. Find the Youngest Person with the Highest Account Balance

In [97]:
query_19 = [
    {"$sort": {  "age": 1, "balance": -1 }},
    {"$limit": 1}
]

result_19 = collection.aggregate(query_19)

list(result_19)

[{'_id': ObjectId('66c45a25e5b03eae919f4bb7'),
  'age': 18,
  'job': 'student',
  'marital': 'single',
  'education': 'primary',
  'default': 'no',
  'balance': 1944,
  'housing': 'no',
  'loan': 'no',
  'contact': 'telephone',
  'day': 10,
  'month': 'aug',
  'duration': 122,
  'campaign': 3,
  'pdays': -1,
  'previous': 0,
  'poutcome': 'unknown',
  'y': 'no'}]

In [106]:
list(collection.find({}, {"age": 1, "balance": 1, "_id": 0}).sort({  "age": 1, "balance": -1 }).limit(1))

[{'age': 18, 'balance': 1944}]

#### 20. Identify Customers with Consistently Low Balances

In [109]:
list(collection.find({"balance": {"$gt": 100}}, {"balance": 1}).sort({"balance": 1 }).limit(10))

[{'_id': ObjectId('66c45a1fe5b03eae919eaf2d'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eb49c'), 'balance': 101},
 {'_id': ObjectId('66c45a20e5b03eae919ebd33'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eb5bc'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eaf6e'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eb9af'), 'balance': 101},
 {'_id': ObjectId('66c45a20e5b03eae919ec6cf'), 'balance': 101},
 {'_id': ObjectId('66c45a20e5b03eae919ec319'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eb3ad'), 'balance': 101},
 {'_id': ObjectId('66c45a1fe5b03eae919eacf6'), 'balance': 101}]

#### 21. Analyze the Relationship Between Campaign Duration and Outcome

In [107]:
# create index for optimization
collection.create_index({"duration": 1, "y": 1})

'duration_1_y_1'

In [115]:
result_21 = collection.aggregate([
    {"$group": {
        "_id": "$y",
        "avgDuration": {"$avg": "$duration"},
        "maxDuration": {"$max": "$duration"},
        "minDuration": {"$min": "$duration"}
    }}
])

list(result_21)

[{'_id': 'no',
  'avgDuration': 221.18280647262162,
  'maxDuration': 4918,
  'minDuration': 0},
 {'_id': 'yes',
  'avgDuration': 537.2945736434109,
  'maxDuration': 3881,
  'minDuration': 8}]

#### 22. Detect Loan Trends Among Different Job Categories

In [117]:
# create index for optimization
collection.create_index({"job": 1, "loan": 1})

'job_1_loan_1'

In [124]:
result_22 = collection.aggregate([
    {"$match": {"loan": "yes"}},
    {"$group": {
        "_id": "$job",
        "countLoans": {"$sum": 1}
    }},
    {"$sort": {"countLoans": -1}}
])

list(result_22)

[{'_id': 'blue-collar', 'countLoans': 1684},
 {'_id': 'technician', 'countLoans': 1309},
 {'_id': 'management', 'countLoans': 1253},
 {'_id': 'admin.', 'countLoans': 991},
 {'_id': 'services', 'countLoans': 836},
 {'_id': 'entrepreneur', 'countLoans': 356},
 {'_id': 'retired', 'countLoans': 309},
 {'_id': 'self-employed', 'countLoans': 229},
 {'_id': 'housemaid', 'countLoans': 152},
 {'_id': 'unemployed', 'countLoans': 109},
 {'_id': 'student', 'countLoans': 12},
 {'_id': 'unknown', 'countLoans': 4}]

- blue-collar workers are taking more loans

#### 23. Identify the Most Common Outcomes for Married Individuals

In [ ]:
result_22 = collection.aggregate([
    {"$group": {
        "_id": "$job",
        "countLoans": {"$sum": 1}
    }},
    {"$sort": {"countLoans": -1}}
])

list(result_22)

#### 24. Find the Distribution of Account Balances Across Different Education Levels

In [ ]:
result_22 = collection.aggregate([
    {"$group": {
        "_id": "$education",
        "countLoans": {"$sum": ""}
    }},
    {"$sort": {"countLoans": -1}}
])

list(result_22)

#### 25. How are account balances distributed among different education levels?

#### 26. Find ranking of Customers based on Account Balance.